<a href="https://colab.research.google.com/github/pgurazada/causal_inference/blob/master/econml_estimators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install -q econml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 49.3 MB/s eta 0:00:00


In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier, XGBRegressor

from econml.metalearners import SLearner, TLearner, XLearner
from econml.dml import SparseLinearDML, CausalForestDML

/usr/local/lib/python3.10/dist-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/usr/local/lib/python3.10/dist-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_size=100, num_

# Data

In [3]:
data_df = pd.read_csv("/content/hillstrom_clean.csv")

In [4]:
data_df.sample(5)

,recency,history,mens,womens,newbie,visit,conversion,spend,zip_code__rural,zip_code__surburban,zip_code__urban,channel__multichannel,channel__phone,channel__web,treatment
45740,12,115.49,0,1,0,0,0,0.0,0,0,1,0,0,1,0
10598,10,29.99,0,1,1,0,0,0.0,1,0,0,0,0,1,2
34103,5,146.67,1,0,1,0,0,0.0,0,0,1,0,1,0,2
41271,1,287.54,0,1,0,0,0,0.0,0,0,1,0,1,0,2
14040,9,29.99,1,0,1,0,0,0.0,0,1,0,0,1,0,0


Historical customer attributes at your disposal include:
- Recency: Months since last purchase.
- History_Segment: Categorization of dollars spent in the past year.
- History: Actual dollar value spent in the past year.
- Mens: 1/0 indicator, 1 = customer purchased Mens merchandise in the past year.
- Womens: 1/0 indicator, 1 = customer purchased Womens merchandise in the past year.
- Zip_Code: Classifies zip code as Urban, Suburban, or Rural. - Newbie: 1/0 indicator, 1 = New customer in the past twelve months. - Channel: Describes the channels the customer purchased from in the past year.
- Treatment: Mens E-Mail, Womens E-Mail, No E-Mail

Finally, we have a series of variables describing activity in the two weeks following delivery of the e-mail campaign:
- Visit: 1/0 indicator, 1 = Customer visited website in the following two weeks.
- Conversion: 1/0 indicator, 1 = Customer purchased merchandise in the following two weeks.
- Spend: Actual dollars spent in the following two weeks.

In [5]:
data_df.visit.describe()

count    64000.000000
mean         0.146781
std          0.353890
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: visit, dtype: float64

In [6]:
data_df.conversion.describe()

count    64000.000000
mean         0.009031
std          0.094604
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: conversion, dtype: float64

# Overall Impact

In [7]:
treatment_map = {
    0: 'control',
    1: 'womens_email',
    2: 'mens_email'
}

In [8]:
# Men's emailer
(
    data_df.query("(treatment == 0 | treatment == 2)")
           .groupby('treatment')
           .agg({'visit': 'mean', 'conversion': 'mean', 'spend': 'mean'})
)

,visit,conversion,spend
treatment,,,
0,0.106167,0.005726,0.652789
2,0.182757,0.012531,1.422617


In [9]:
# Women's emailer
(
    data_df.query("(treatment == 0 | treatment == 1)")
           .groupby('treatment')
           .agg({'visit': 'mean', 'conversion': 'mean', 'spend': 'mean'})
)

,visit,conversion,spend
treatment,,,
0,0.106167,0.005726,0.652789
1,0.151400,0.008837,1.077202


# CATE

## Estimating Base Learners

In [10]:
model_regression = XGBRegressor()
model_classification = XGBClassifier()

## S-Learner

Estimated CATE:

$$
\hat{\tau}(x) = E[Y|X=x, T=1]-E[Y|X=x, T=0]=\hat{\mu}(x, 1) - \hat{\mu}(x, 0)
$$

where $\hat{\mu}=M(Y\sim(X, T))$ is any machine learning algorithm that is estimated on training data.

*Visits*

In [11]:
slearner_visit = SLearner(overall_model=model_classification)

In [12]:
target = 'visit'
treatment = 'treatment'

In [13]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_visit = data_df[target]
T = data_df[treatment]

In [14]:
X_train, X_test, y_visit_train, y_visit_test, T_train, T_test = train_test_split(
    X, y_visit, T, test_size=0.5, random_state=42
)

In [15]:
slearner_visit.fit(y_visit_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [16]:
slearner_visit.const_marginal_ate(X_test)

array([0.00403125, 0.01115625])

In [17]:
slearner_visit.ate(X_test, T0=0, T1=1)

0.00403125

In [18]:
slearner_visit.ate(X_test, T0=0, T1=2)

0.01115625

In [19]:
slearner_visit.ate(X_test, T0=1, T1=2).mean()

0.007125

*Spends*

In [20]:
target = 'spend'
treatment = 'treatment'

In [21]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_spend = data_df[target]
T = data_df[treatment]

In [22]:
X_train, X_test, y_spend_train, y_spend_test, T_train, T_test = train_test_split(
    X, y_spend,T, test_size=0.3, random_state=42
)

In [23]:
slearner_spend = SLearner(overall_model=model_regression)

In [24]:
slearner_spend.fit(y_spend_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [25]:
slearner_spend.const_marginal_ate(X_test)

array([0.41212702, 0.57304573], dtype=float32)

In [26]:
slearner_spend.ate(X_test, T0=0, T1=1)

0.4121282289664183

In [27]:
slearner_spend.ate(X_test, T0=0, T1=2)

0.5730449149145473

In [28]:
slearner_spend.ate(X_test, T0=1, T1=2)

0.16091668594812897

##T-Learner

Estimated CATE:

$$
\hat{\tau}(x) = E[Y|X=x, T=1]-E[Y|X=x, T=0]=\hat{\mu}_1(x, 1) - \hat{\mu}_0(x, 0)
$$

where $\hat{\mu}_0=M_0(Y^0 \sim X^0)$, $\hat{\mu}_1=M_1(Y^1 \sim X^1)$ are any machine learning algorithms that are estimated on control and treatment subsets of training data respectively.

*Visits*

In [29]:
tlearner_visit = TLearner(models=model_classification)

In [30]:
target = 'visit'
treatment = 'treatment'

In [31]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_visit = data_df[target]
T = data_df[treatment]

In [32]:
X_train, X_test, y_visit_train, y_visit_test, T_train, T_test = train_test_split(
    X, y_visit, T, test_size=0.3, random_state=42
)

In [33]:
tlearner_visit.fit(y_visit_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [34]:
tlearner_visit.models

[XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categ

In [35]:
tlearner_visit.const_marginal_ate(X_test)

array([0.00744792, 0.0178125 ])

In [36]:
tlearner_visit.ate(X_test, T0=0, T1=1)

0.007447916666666667

In [37]:
tlearner_visit.ate(X_test, T0=0, T1=2)

0.0178125

In [38]:
tlearner_visit.ate(X_test, T0=1, T1=2)

0.010364583333333333

*Spends*

In [39]:
target = 'spend'
treatment = 'treatment'

In [40]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_spend = data_df[target]
T = data_df[treatment]

In [41]:
X_train, X_test, y_spend_train, y_spend_test, T_train, T_test = train_test_split(
    X, y_spend, T, test_size=0.3, random_state=42
)

In [42]:
tlearner_spend = TLearner(models=model_regression)

In [43]:
tlearner_spend.fit(y_spend_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [44]:
tlearner_spend.const_marginal_ate(X_test)

array([0.5396046, 0.7586634], dtype=float32)

In [45]:
tlearner_spend.ate(X_test, T0=0, T1=1)

0.5396059393680237

In [46]:
tlearner_spend.ate(X_test, T0=0, T1=2)

0.7586630381449989

In [47]:
tlearner_spend.ate(X_test, T0=1, T1=2)

0.21905709877697518

##X-Learner

Estimated CATE:

$\hat{\mu}_0=M_0(Y^0 \sim X^0), \hat{\mu}_1=M_1(Y^1 \sim X^1)$

$\hat{D}^1 = Y^1 - \mu_0(X^1), \hat{D}^0 = \mu_1(X^0) - Y^0$

$\hat{\tau}_0 = M_3(\hat{D}^0 \sim X^0), \hat{\tau}_1 = M_4(\hat{D}^1 \sim X^1)$

$\hat{\tau}(x) = g(x)\hat{\tau}_0(x) + (1-g(x))\hat{\tau}_1(x)$

Where $M_1, M_2$ are any machine learning models to estimate the treatment and control outcomes & $M_3 \& M_4$ are any machine learning models to estimate the residuals. $g(x)$ is a propensity model that is used to weigh the CATT and CATC.



*Visits*

In [48]:
xlearner_visit = XLearner(
    models=model_classification,
    cate_models=model_regression
)

In [49]:
target = 'visit'
treatment = 'treatment'

In [50]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_visit = data_df[target]
T = data_df[treatment]

In [51]:
X_train, X_test, y_visit_train, y_visit_test, T_train, T_test = train_test_split(
    X, y_visit, T, test_size=0.3, random_state=42
)

In [52]:
xlearner_visit.fit(y_visit_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [53]:
xlearner_visit.const_marginal_ate(X_test)

array([0.02350839, 0.04534053])

In [56]:
xlearner_visit.ate(X_test, T0=0, T1=1)

0.023508385490667687

In [57]:
xlearner_visit.ate(X_test, T0=0, T1=2)

0.04534053351298643

In [58]:
xlearner_visit.ate(X_test, T0=1, T1=2).mean()

0.02183214802231875

*Spends*

In [59]:
target = 'spend'
treatment = 'treatment'

In [60]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_spend = data_df[target]
T = data_df[treatment]

In [61]:
X_train, X_test, y_spend_train, y_spend_test, T_train, T_test = train_test_split(
    X, y_spend, T, test_size=0.3, random_state=42
)

In [62]:
xlearner_spend = XLearner(
    models=model_regression,
    cate_models=model_regression
)

In [63]:
xlearner_spend.fit(y_spend_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [64]:
xlearner_spend.const_marginal_ate(X_test)

array([0.47082475, 0.81201032])

In [65]:
xlearner_spend.ate(X_test, T0=0, T1=1)

0.47082475450005423

In [66]:
xlearner_spend.ate(X_test, T0=0, T1=2)

0.8120103176445745

# Double Machine Learning

We formulate the problem like so:

$Y = \theta(X).T+g(X, W) + ϵ, E[\epsilon|X, W] = 0 \tag{1}$

$\Longrightarrow E[Y|X, W]=E[\theta(X).T|X, W] + E[g(X, W)|X, W] + E[\epsilon|X, W] \tag{2}$

$\Longrightarrow E[Y|X, W] = \theta(X).E[T|X, W] + g(X, W) \tag{3}$

Subtracting equation $(1)$ and $(3)$, we get:

$Y - E[Y|X, W] = \theta(X)(T - E[T|X, W]) + \epsilon$

Denoting $\tilde{Y} = Y - E[Y|X, W]$ and $\tilde{T} = T - E[T|X, W]$, we get:

$\tilde{Y} = \theta(X).\tilde{T} + \epsilon$

## Linear Double Machine Learning (DML)

*Visits*

In [67]:
dml_learner_visit = SparseLinearDML(
    model_y=model_classification,
    model_t=model_classification,
    linear_first_stages=False,
    discrete_treatment=True
)

In [68]:
target = 'visit'
treatment = 'treatment'

In [69]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_visit = data_df[target]
T = data_df[treatment]

In [70]:
X_train, X_test, y_visit_train, y_visit_test, T_train, T_test = train_test_split(
    X, y_visit, T, test_size=0.3, random_state=42
)

In [71]:
dml_learner_visit.fit(y_visit_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [72]:
dml_learner_visit.const_marginal_ate(X_test)

array([0.04463286, 0.15787598])

In [73]:
dml_learner_visit.ate(X_test, T0=0, T1=1)

0.0446328585809648

In [74]:
dml_learner_visit.ate(X_test, T0=0, T1=2)

0.15787597641189727

In [75]:
dml_learner_visit.ate(X_test, T0=1, T1=2)

0.11324311783093245

*Spends*

In [76]:
target = 'spend'
treatment = 'treatment'

In [77]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_spend = data_df[target]
T = data_df[treatment]

In [78]:
X_train, X_test, y_spend_train, y_spend_test, T_train, T_test = train_test_split(
    X, y_spend, T, test_size=0.3, random_state=42
)

In [79]:
dml_learner_spend = SparseLinearDML(
    model_y=model_regression,
    model_t=model_classification,
    linear_first_stages=False,
    discrete_treatment=True
)

In [80]:
dml_learner_spend.fit(y_spend_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [81]:
dml_learner_spend.const_marginal_ate(X_test)

array([0.01423481, 2.16032402])

In [82]:
dml_learner_spend.ate(X_test, T0=0, T1=1)

0.014234806227998806

In [83]:
dml_learner_spend.ate(X_test, T0=0, T1=2)

2.160324016016989

In [84]:
dml_learner_spend.ate(X_test, T0=1, T1=2)

2.146089209788991

## Causal Forest Double Machine Learning (DML)

*Visits*

In [85]:
dml_learner_visit = CausalForestDML(
    model_y=model_classification,
    model_t=model_classification,
    discrete_treatment=True
)

In [86]:
target = 'visit'
treatment = 'treatment'

In [87]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_visit = data_df[target]
T = data_df[treatment]

In [88]:
X_train, X_test, y_visit_train, y_visit_test, T_train, T_test = train_test_split(
    X, y_visit, T, test_size=0.3, random_state=42
)

In [89]:
dml_learner_visit.fit(y_visit_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [90]:
dml_learner_visit.const_marginal_ate(X_test)

array([0.03984612, 0.06958578])

In [91]:
dml_learner_visit.ate(X_test, T0=0, T1=1)

0.03984611955001916

In [92]:
dml_learner_visit.ate(X_test, T0=0, T1=2)

0.06958577864190067

In [93]:
dml_learner_visit.ate(X_test, T0=1, T1=2)

0.029739659091881507

*Spends*

In [94]:
target = 'spend'
treatment = 'treatment'

In [95]:
X = data_df.drop(columns=['visit', 'conversion', 'spend', 'treatment'])
y_spend = data_df[target]
T = data_df[treatment]

In [96]:
X_train, X_test, y_spend_train, y_spend_test, T_train, T_test = train_test_split(
    X, y_spend, T, test_size=0.3, random_state=42
)

In [97]:
dml_learner_spend = CausalForestDML(
    model_y=model_regression,
    model_t=model_classification,
    discrete_treatment=True
)

In [98]:
dml_learner_spend.fit(y_spend_train, T_train, X=X_train)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [99]:
dml_learner_spend.const_marginal_ate(X_test)

array([0.49140381, 0.73541033])

In [100]:
dml_learner_spend.ate(X_test, T0=0, T1=1)

0.49140381380938547

In [101]:
dml_learner_spend.ate(X_test, T0=0, T1=2)

0.7354103267578481

In [102]:
dml_learner_spend.ate(X_test, T0=1, T1=2)

0.24400651294846265